In [ ]:
! uv pip install drytorch

In [1]:
from drytorch import Experiment, init_trackers


init_trackers(mode='minimal')

tuning_experiment = Experiment(
    None, name='TuningExperiment', par_dir='experiments'
)
with tuning_experiment.create_run():
    pass  # no logging output

init_trackers()
standard_experiment = Experiment(
    None, name='StandardExperiment', par_dir='experiments'
)
with standard_experiment.create_run():
    pass  # log experiment's name

[2026-01-06 17:27:49] - Running experiment: StandardExperiment.


In [2]:
import pathlib

from drytorch import extend_default_trackers
from drytorch.trackers.csv import CSVDumper


csv_dumper = CSVDumper(par_dir=pathlib.Path('experiments', 'my_directory'))
extend_default_trackers([csv_dumper])

In [3]:
csv_dumper_experiment = Experiment(
    None, name='ExperimentWithCSVDumper', par_dir='experiments'
)

with csv_dumper_experiment.create_run():
    if CSVDumper.get_current() is not csv_dumper:
        raise AssertionError('Trackers should coincide.')

[2026-01-06 17:27:49] - Running experiment: ExperimentWithCSVDumper.


In [4]:
import functools
import pprint

from typing_extensions import override

from drytorch.core import log_events
from drytorch.core.track import Tracker


class MetadataVisualizer(Tracker):
    """Tracker that prints the metadata on the console."""

    @functools.singledispatchmethod
    @override
    def notify(self, event: log_events.Event) -> None:
        return super().notify(event)

    @notify.register
    def _(self, event: log_events.ModelRegistrationEvent) -> None:
        pprint.pp(event.architecture_repr)
        return super().notify(event)

    @notify.register
    def _(self, event: log_events.ActorRegistrationEvent) -> None:
        pprint.pp(event.metadata)
        return super().notify(event)